# Entity Framework Core Examples

Making use of notebooks to printout sql queries formed by EF
When in doubt, make use of notebook to create EF expression and print out the query string.
This can help when you need to optimize the query for performance.

Issues:
- "using var ctx..." gives error when "using" is used

In [58]:
#r "nuget: Microsoft.EntityFrameworkCore.SqlServer, 7.0.5"

using Microsoft.EntityFrameworkCore;
using System;

public class ItemContext : DbContext
{
	public DbSet<Item> Items { get; set; }
	public DbSet<Tag> Tags { get; set; }

	protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder)
	{
		// use a valid connection string here so the parser doesn't complain
		optionsBuilder.UseSqlServer("Server=(localdb)\\MSSQLLocalDB;Database=SomeDbName;Trusted_Connection=True");
	}
}

public class Item
{
	public int Id { get; set; }
	public string Name { get; set; } = string.Empty;
	public string Status { get; set; } = string.Empty;
	public int SomeId { get; set; }
	public DateTimeOffset Created { get; set; }
	public string EnumValue { get; set; } = string.Empty;
}

public class Tag
{
	public int Id { get; set; }
	public string Name { get; set; } = string.Empty;
	public string Value { get; set; } = string.Empty;
}

Installed Packages Microsoft.EntityFrameworkCore.SqlServer, 7.0.5

<br><br><br><br>

## SELECT 
Examples to show how EF forms SQL queries

In [59]:
var ctx = new ItemContext();
IQueryable<Item> query = ctx.Items;

// as long as we do not call ToList or FirstOrDefault
// the following LINQ will not attempt to establish
// connection to database defined in connection string

var itemId = 10;
query = ctx.Items.Where(item => item.Id > itemId);
Console.WriteLine(query.ToQueryString());

DECLARE @__itemId_0 int = 10;

SELECT [i].[Id], [i].[Created], [i].[EnumValue], [i].[Name], [i].[SomeId], [i].[Status]
FROM [Items] AS [i]
WHERE [i].[Id] > @__itemId_0


In [60]:
// when we use raw values in the expression, we do not get parameterized queries
// compare with the following:
query = ctx.Items.Where(item => item.Id > 10);
Console.WriteLine(query.ToQueryString());

SELECT [i].[Id], [i].[Created], [i].[EnumValue], [i].[Name], [i].[SomeId], [i].[Status]
FROM [Items] AS [i]
WHERE [i].[Id] > 10


In [61]:
var nameToMatch = "HELLO WORLD";
query = ctx.Items.Where(item => item.Name.ToUpper().Contains(nameToMatch));
Console.WriteLine(query.ToQueryString());

DECLARE @__nameToMatch_0 nvarchar(4000) = N'HELLO WORLD';

SELECT [i].[Id], [i].[Created], [i].[EnumValue], [i].[Name], [i].[SomeId], [i].[Status]
FROM [Items] AS [i]
WHERE (@__nameToMatch_0 LIKE N'') OR CHARINDEX(@__nameToMatch_0, UPPER([i].[Name])) > 0


In [62]:
query = ctx.Items.Where(item => item.Created < DateTime.Now && item.Id == itemId);
Console.WriteLine(query.ToQueryString());

DECLARE @___itemId_0 int = 10;

SELECT [i].[Id], [i].[Created], [i].[EnumValue], [i].[Name], [i].[SomeId], [i].[Status]
FROM [Items] AS [i]
WHERE [i].[Created] < CAST(GETDATE() AS datetimeoffset) AND [i].[Id] = @___itemId_0


In [63]:
// break up Where clauses -> doesn't seem to affect SQL generated
query = ctx.Items.Where(item => item.Created < DateTime.Now).Where(item => item.Id == 300);
Console.WriteLine(query.ToQueryString());

SELECT [i].[Id], [i].[Created], [i].[EnumValue], [i].[Name], [i].[SomeId], [i].[Status]
FROM [Items] AS [i]
WHERE [i].[Created] < CAST(GETDATE() AS datetimeoffset) AND [i].[Id] = 300


This example shows why it's a bad idea to use a large collection with Contains(...)

In [64]:
var list = Enumerable.Range(1, 20).Select(x => x.ToString());
query = ctx.Items.Where(item => list.Contains(item.Name));
Console.WriteLine(query.ToQueryString());

SELECT [i].[Id], [i].[Created], [i].[EnumValue], [i].[Name], [i].[SomeId], [i].[Status]
FROM [Items] AS [i]
WHERE [i].[Name] IN (N'1', N'2', N'3', N'4', N'5', N'6', N'7', N'8', N'9', N'10', N'11', N'12', N'13', N'14', N'15', N'16', N'17', N'18', N'19', N'20')


<br><br><br><br>

## SELECT + DISTINCT + ORDER BY

In [65]:
// if we have logging framework added, entity framework
// will complain about the calling order of Distinct + OrderBy
// the generated query will not show "ORDER BY"
var wrongQuery = ctx.Items
    .Select(item => new { item.Name, item.Created })
    .OrderBy(it => it.Created)
    .Distinct();

Console.WriteLine("Wrong ORDER BY + DISTINCT -- the ORDER BY is missing in SQL query");
Console.WriteLine(wrongQuery.ToQueryString());
Console.WriteLine();

// the SQL generated will show "ORDER BY" but
// as of Mar 2023, the generated SQL doesn't seem to be "right"
// the following would look more "right"
// SELECT DISTINCT ...
// FROM TABLE
// ORDER BY Created
var correctQuery = ctx.Items
    .Select(item => new { item.Name, item.Created })
    .Distinct()
    .OrderBy(it => it.Created);

Console.WriteLine("Correct DISTINCT + ORDER BY -- looks more complicated but ORDER BY is observed in SQL query");
Console.WriteLine(correctQuery.ToQueryString());


Wrong ORDER BY + DISTINCT -- the ORDER BY is missing in SQL query
SELECT DISTINCT [i].[Name], [i].[Created]
FROM [Items] AS [i]

Correct DISTINCT + ORDER BY -- looks more complicated but ORDER BY is observed in SQL query
SELECT [t].[Name], [t].[Created]
FROM (
    SELECT DISTINCT [i].[Name], [i].[Created]
    FROM [Items] AS [i]
) AS [t]
ORDER BY [t].[Created]


In [66]:
// this query demonstrate query that requires a custom order by
// make sure to double check query generated
// there used to be a bug with how the query was generated in
// earlier versions of entity framework
var query = ctx.Items
    .Select(item => new
    {
        item,
        sortIndex =
            item.EnumValue == "Pending" ? 0 :
            item.EnumValue == "Accepted" ? 1 :
            item.EnumValue == "Rejected" ? 2 : 3
    })
    .OrderBy(anon => anon.sortIndex)
    .Select(anon => anon.item);

Console.WriteLine(query.ToQueryString());

SELECT [i].[Id], [i].[Created], [i].[EnumValue], [i].[Name], [i].[SomeId], [i].[Status]
FROM [Items] AS [i]
ORDER BY CASE
    WHEN [i].[EnumValue] = N'Pending' THEN 0
    WHEN [i].[EnumValue] = N'Accepted' THEN 1
    WHEN [i].[EnumValue] = N'Rejected' THEN 2
    ELSE 3
END


<br><br><br><br>

## JOIN

In [67]:
// example of left join
var joinQuery = from item in ctx.Items
                join tag in ctx.Tags
                on item.Name equals tag.Name into tagged
                from t in tagged.DefaultIfEmpty()
                select new { item.Name, t.Value };

Console.WriteLine(joinQuery.ToQueryString());

SELECT [i].[Name], [t].[Value]
FROM [Items] AS [i]
LEFT JOIN [Tags] AS [t] ON [i].[Name] = [t].[Name]
